In [18]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/flyhopper/scripts/Julia/Project.toml`


In [19]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using StaticArrays

In [20]:
ℓ1 = 1.0
ℓ2 = 1.0
ℓc1 = ℓ1/2
ℓc2 = ℓ2/2
m1 = 1.0
m2 = 1.0
g = 9.81
I1 = m1*(ℓ1^2)
I2 = m2*(ℓ2^2)

1.0

In [21]:
g = -9.81 # gravitational acceleration in z-direction
world = RigidBody{Float64}("world")
doublependulum = Mechanism(world; gravity = SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

In [23]:
axis = SVector(0., 1., 0.) # joint axis
m = zeros(3, 3)
inertia1 = SpatialInertia(CartesianFrame3D("frame0"),
    moment=SArray{Tuple{3,3}}(m),
    com=SVector(0, 0, ℓc1),
    mass=m1)

link0 = RigidBody(inertia1)
joint0 = Joint("joint0", Revolute(axis))
before_joint0_to_world = one(Transform3D, frame_before(joint0), default_frame(world))

attach!(doublependulum, world, link0, joint0, joint_pose = before_joint0_to_world)

inertia2 = SpatialInertia(CartesianFrame3D("frame1"),
    moment=SArray{Tuple{3,3}}(m),
    com=SVector(0, 0, ℓc2),
    mass=m2)

link1 = RigidBody(inertia2)
joint1 = Joint("joint1", Revolute(axis))
before_joint1_to_after_joint0 = Transform3D(frame_before(joint1), frame_after(joint0), SVector(0, 0, ℓ1))
attach!(doublependulum, link0, link1, joint1, joint_pose = before_joint1_to_after_joint0)

Spanning tree:
Vertex: world (root)
  Vertex: frame0, Edge: joint0
    Vertex: frame1, Edge: joint1
No non-tree joints.

In [68]:
state = MechanismState(doublependulum)
relative_transform(state, default_frame(link1), default_frame(link0))

Transform3D from "after_joint1" to "after_joint0":
rotation: 0.0 rad about [1.0, 0.0, 0.0], translation: [0.0, 0.0, 1.0]

In [25]:
#=
curdir = pwd()
urdfpath = joinpath(curdir, "../../res/flyhopper_mockup/urdf/flyhopper_mockup_jl.urdf")
bot = parse_urdf(urdfpath, floating=False)
# remove_fixed_tree_joints!(bot);
g = -9.81 # gravitational acceleration in z-direction
state = MechanismState(bot)
set_configuration!(state, [1; 0; 0; 0; 0; 0; 0.7])
q = configuration(state)
v = velocity(state)
ts, qs, vs = simulate(state, 5., Δt = 1e-3);
=#

In [26]:
function f(x)
    # double pendulum dynamics
    
    q1 = x[1]
    q1̇ = x[2]
    q2 = x[3]
    q2̇ = x[4]
    
    M = zeros(2, 2)
    M[1, 1] = I1 + I2 + m2*ℓ1^2 + 2*m2*ℓ1*ℓc2*cos(q2)
    M[1, 2] = I2 + m2*ℓ1*ℓc2*cos(q2)
    M[2, 1] = I2 + m2*ℓ1*ℓc2*cos(q2)
    M[2, 2] = I2
    
    C = zeros(2, 2)
    C[1, 1] = -2*m2*ℓ1*ℓc2*sin(q2)*q2̇ 
    C[1, 2] = -m2*ℓ1*ℓc2*sin(q2)*q2̇ 
    C[2, 1] = m2*ℓ1*ℓc2*sin(q2)*q1̇
    C[2, 2] = 0
    
    τg = zeros(2)
    τg[1] = -m1*g*ℓc1*sin(q1) - m2*g*(ℓ1*sin(q1) + ℓc2*sin(q1+q2))
    τg[2] = -m2*g*ℓc2*sin(q1+q2)
    q_d = zeros(2)
    q_d[1] = q1̇ 
    q_d[2] = q2̇ 
    q_dd = inv(M)*(τg - C*q_d)
    q1̈  = q_dd[1]
    q2̈  = q_dd[2]
    
    ẋ = zeros(4)
    ẋ[1] = q1̇  # q1 dot
    ẋ[2] = q1̈  # q1 double dot
    ẋ[3] = q2̇  # q2 dot
    ẋ[4] = q2̈  # q2 double dot
    
end

f (generic function with 1 method)

In [27]:
function euler_step(xk)
    xn = xk .+ h*f(xk)
end

euler_step (generic function with 1 method)

In [28]:
function simulate!(xtraj, N)
    for k = 1:(N-1)
        xtraj[:,k+1] .= euler_step(xtraj[:,k])
    end
end

simulate! (generic function with 1 method)

In [29]:
Tf = 10.0
h = 0.001 #20 Hz
N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1));

In [30]:
x0 = [5*(pi/180); 0.0; 0.0; 0.0]
xtraj = zeros(4,N)
xtraj[:,1] = x0;

In [31]:
simulate!(xtraj, N)

In [57]:

# set_configuration!(state, link1, 0.4)
# set_velocity!(state, link0, 1.)
# set_velocity!(state, link1, 2.);

In [58]:
q = configuration(state)
v = velocity(state)

2-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0

In [59]:
q1 = xtraj[1, :]
q2 = xtraj[3, :]
qs = convert(AbstractArray{Float64, 2}, [q1 q2]) 
ts = convert(AbstractVector{Float64}, thist) # AbstractVector(thist)
q_array = [ qs[i,:] for i in 1:size(qs,1)] 

10001-element Vector{Vector{Float64}}:
 [0.08726646259971647, 0.0]
 [0.08677789240751389, -0.0004885701922025865]
 [0.08628864001108741, -0.000977822588629076]
 [0.08579870423471148, -0.001467758365005009]
 [0.08530808390652234, -0.001958378693194147]
 [0.08481677785857454, -0.002449684741141941]
 [0.08432478492689789, -0.0029416776728185884]
 [0.08383210395155481, -0.003434358648161668]
 [0.08333873377669812, -0.003927728823018362]
 [0.08284467325062923, -0.00442178934908725]
 [0.0823499212258568, -0.004916541373859692]
 [0.08185447655915572, -0.005411986040560779]
 [0.08135833811162663, -0.005908124488089868]
 ⋮
 [-0.47203843971833637, -0.5593049023180523]
 [-0.47203843971833637, -0.5593049023180523]
 [-0.47203843971833637, -0.5593049023180523]
 [-0.47203843971833637, -0.5593049023180523]
 [-0.47203843971833637, -0.5593049023180523]
 [-0.47203843971833637, -0.5593049023180523]
 [-0.47203843971833637, -0.5593049023180523]
 [-0.47203843971833637, -0.5593049023180523]
 [-0.4720384397183

In [71]:
mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true, inertias=false));

render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8710
└ @ MeshCat /home/ben/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("before_joint0" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x65, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x30], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "frame0" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x66, 0x72, 0x61, 0x6d, 0x65, 0x30], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("frame1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6d, 0x65, 0x30, 0x2f, 0x66, 0x72, 0x61, 0x6d, 0x65, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "before_joint1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x65, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "after_joint0" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x30], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))))), Set{Any}(), ip"127.0.0.1", 8710))

In [66]:
set_configuration!(mvis, [x0[1], x0[3]])

In [61]:
# Now we can simply call `simulate`, which will return a tuple consisting of:
# * simulation times (a `Vector` of numbers)
# * joint configuration vectors (a `Vector` of `Vector`s)
# * joint velocity vectors (a `Vector` of `Vector`s)

# ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);
# And animate:
# MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);
using MeshCat
animation = Animation(mvis, ts, q_array)
setanimation!(mvis, animation)